# <font color='blue'>Data Science Challenge @ ITA 2022</font>
# <font color='blue'>Equipe DIOMGIS</font>

## <font color='blue'>1º Fase</font>

### <font color='blue'>Predição de pregões futuros de ativos que compõem o índice SP500.</font>

![title](data\image\logo.jpeg)

In [15]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


In [16]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.

!pip install -q -U watermark

In [17]:
# Bibliotecas e Frameworks

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import *
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from keras.losses import MeanSquaredError
from tensorboard import notebook
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from time import time
from datetime import datetime
import os

In [18]:
# Versões dos pacotes usados neste jupyter notebook

%reload_ext watermark
%watermark -a "Equipe DIOMGIS" --iversions

Author: Equipe DIOMGIS

pandas           : 1.4.2
pandas_datareader: 0.10.0
tensorflow       : 2.10.0
seaborn          : 0.11.2
keras            : 2.10.0
tensorboard      : 2.10.0
matplotlib       : 3.5.1
numpy            : 1.22.3



In [22]:
# Parametros fixos

verbose = 2
seed = 25
steps = 30
epochs = 1000
batch_size = 32
logRetPeriod = 20
graphic = False
downloadData = False
trainModel = False

In [23]:
forecast = ['2022-10-24', '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', 
            '2022-10-31', '2022-11-01', '2022-11-02', '2022-11-03', '2022-11-04', 
            '2022-11-07', '2022-11-08', '2022-11-09', '2022-11-10', '2022-11-11',
            '2022-11-14', '2022-11-15', '2022-11-16', '2022-11-17', '2022-11-18']

In [19]:
sns.set_style('whitegrid')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.rcParams['figure.figsize'] = (15, 10)
np.random.seed(seed)

In [20]:
#Confirma se o TensorFlow pode acessar a GPU

device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
    
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [21]:
# Estado da GPU

!nvidia-smi

Wed Oct 19 13:39:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:65:00.0  On |                  N/A |
|  0%   39C    P2    27W / 220W |   1185MiB /  8192MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
setores = {
    'Industrials': ['MMM', 'AOS', 'ALK', 'ALLE', 'AAL', 'AME', 'BA',
                    'CHRW', 'CARR', 'CAT', 'CTAS', 'CPRT', 'CSGP',
                    'CSX', 'CMI', 'DE', 'DAL', 'DOV', 'ETN', 'EMR',
                    'EFX', 'EXPD', 'FAST', 'FDX', 'FTV', 'FBHS',
                    'GNRC', 'GD', 'GE', 'HON', 'HWM', 'HII', 'IEX',
                    'ITW', 'IR', 'JBHT', 'J', 'JCI', 'LHX', 'LDOS',
                    'LMT', 'MAS', 'NDSN', 'NSC', 'NOC', 'ODFL',
                    'OTIS', 'PCAR', 'PH', 'PNR', 'PWR', 'RTX', 'RSG',
                    'RHI', 'ROK', 'ROL', 'SNA', 'LUV', 'SWK', 'TXT',
                    'TT', 'TDG', 'UNP', 'UAL', 'UPS', 'URI', 'VRSK',
                    'WAB', 'WM', 'GWW', 'XYL'],
    
    'HealthCare': ['ABT', 'ABBV', 'ABMD', 'A', 'ALGN', 'ABC', 'AMGN',
                   'BAX', 'BDX', 'BIO', 'TECH', 'BIIB', 'BSX', 'BMY',
                   'CAH', 'CTLT', 'CNC', 'CRL', 'CI', 'COO', 'CVS',
                   'DHR', 'DVA', 'XRAY', 'DXCM', 'EW', 'ELV', 'LLY',
                   'GILD', 'HCA', 'HSIC', 'HOLX', 'HUM', 'IDXX',
                   'ILMN', 'INCY', 'ISRG', 'IQV', 'JNJ', 'LH', 'MCK',
                   'MDT', 'MRK', 'MTD', 'MRNA', 'MOH', 'OGN', 'PKI',
                   'PFE', 'DGX', 'REGN', 'RMD', 'STE', 'SYK', 'TFX',
                   'TMO', 'UNH', 'UHS', 'VRTX', 'VTRS', 'WAT',
                   'WST', 'ZBH', 'ZTS'],
    
    'InformationTechnology': ['ACN', 'ADBE', 'ADP', 'AKAM', 'AMD', 'APH',
                              'ADI', 'ANSS', 'AAPL', 'AMAT', 'ANET', 'ADSK',
                              'AVGO', 'BR', 'CDNS', 'CDW', 'CDAY', 'CSCO',
                              'CTSH', 'GLW', 'DXC', 'ENPH', 'EPAM', 'FFIV',
                              'FIS', 'FISV', 'FLT', 'FTNT', 'IT', 'GPN',
                              'HPE', 'HPQ', 'IBM', 'INTC', 'INTU', 'JKHY',
                              'JNPR', 'KEYS', 'KLAC', 'LRCX', 'MA', 'MCHP',
                              'MU', 'MSFT', 'MPWR', 'MSI', 'NTAP', 'NLOK',
                              'NVDA', 'NXPI', 'ON', 'ORCL', 'PAYX', 'PAYC',
                              'PYPL', 'PTC', 'QRVO', 'QCOM', 'ROP', 'CRM',
                              'STX', 'NOW', 'SWKS', 'SEDG', 'SNPS', 'TEL',
                              'TDY', 'TER', 'TXN', 'TRMB', 'TYL', 'VRSN',
                              'V', 'WDC', 'ZBRA'],
    
    'CommunicationServices': ['ATVI', 'GOOGL', 'GOOG', 'T', 'CHTR', 'CMCSA',
                              'DISH', 'DIS', 'EA', 'FOXA', 'FOX', 'IPG', 'LYV',
                              'LUMN', 'MTCH', 'META', 'NFLX', 'NWSA', 'NWS',
                              'OMC', 'PARA', 'TMUS', 'TTWO', 'TWTR', 'VZ', 'WBD'],
    
    'ConsumerStaples': ['ADM', 'MO', 'BF.B', 'CPB', 'CHD', 'CLX', 'KO', 'CL',
                        'CAG', 'STZ', 'COST', 'EL', 'GIS', 'HSY', 'HRL', 'K',
                        'KDP', 'KMB', 'KHC', 'KR', 'LW', 'MKC', 'TAP', 'MDLZ',
                        'MNST', 'PEP', 'PM', 'PG', 'SJM', 'SYY', 'TSN', 'WBA',
                        'WMT'],
    
    'ConsumerDiscretionary': ['AAP', 'AMZN', 'APTV', 'AZO', 'BBWI',
                              'BBY', 'BKNG', 'BWA', 'CZR', 'KMX', 'CCL',
                              'CMG', 'DHI', 'DRI', 'DG', 'DLTR',
                              'DPZ', 'EBAY', 'ETSY', 'EXPE', 'F',
                              'GRMN', 'GM', 'GPC', 'HAS', 'HLT', 'HD',
                              'LVS', 'LEN', 'LKQ', 'LOW', 'MAR', 'MCD',
                              'MGM', 'MHK', 'NWL', 'NKE', 'NCLH', 'NVR',
                              'ORLY', 'POOL', 'PHM', 'RL', 'ROST', 'RCL',
                              'SBUX', 'TPR', 'TGT', 'TSLA', 'TJX',
                              'TSCO', 'ULTA', 'VFC', 'WHR', 'WYNN', 'YUM'],
    
    'Utilities': ['AES', 'LNT', 'AEE', 'AEP', 'AWK', 'ATO', 'CNP',
                  'CMS', 'ED', 'CEG', 'D', 'DTE', 'DUK', 'EIX',
                  'ETR', 'EVRG', 'ES', 'EXC', 'FE', 'NEE', 'NI',
                  'NRG', 'PCG', 'PNW', 'PPL', 'PEG', 'SRE', 'SO',
                  'WEC', 'XEL'],
    
    'Financials': ['AFL', 'ALL', 'AXP', 'AIG', 'AMP', 'AON', 'AJG',
                   'AIZ', 'BAC', 'WRB', 'BRK.B', 'BLK', 'BK', 'BRO',
                   'COF', 'CBOE', 'SCHW', 'CB', 'CINF', 'C', 'CFG',
                   'CME', 'CMA', 'DFS', 'RE', 'FDS', 'FITB', 'FRC',
                   'BEN', 'GL', 'GS', 'HIG', 'HBAN', 'ICE', 'IVZ',
                   'JPM', 'KEY', 'LNC', 'L', 'MTB', 'MKTX', 'MMC',
                   'MET', 'MCO', 'MS', 'MSCI', 'NDAQ', 'NTRS', 'PNC',
                   'PFG', 'PGR', 'PRU', 'RJF', 'RF', 'SPGI', 'SBNY',
                   'STT', 'SIVB', 'SYF', 'TROW', 'TRV', 'TFC',
                   'USB', 'WFC', 'WTW', 'ZION',  'NLSN'],
    
    'Materials': ['APD', 'ALB', 'AMCR', 'AVY', 'BALL', 'CE', 'CF',
                  'CTVA', 'DOW', 'DD', 'EMN', 'ECL', 'FMC', 'FCX',
                  'IP', 'IFF', 'LIN', 'LYB', 'MLM', 'MOS', 'NEM',
                  'NUE', 'PKG', 'PPG', 'SEE', 'SHW', 'VMC', 'WRK'],
    
    'RealEstate': ['ARE', 'AMT', 'AVB', 'BXP', 'CPT', 'CBRE', 'CCI',
                   'DLR', 'EQIX', 'EQR', 'ESS', 'EXR', 'FRT', 'PEAK',
                   'HST', 'INVH', 'IRM', 'KIM', 'MAA', 'PLD', 'PSA',
                   'O', 'REG', 'SBAC', 'SPG', 'UDR', 'VTR', 'VICI',
                   'VNO', 'WELL', 'WY'],
    
    'Energy': ['APA','BKR', 'CVX', 'COP', 'CTRA', 'DVN', 'FANG', 'EOG',
               'EQT', 'XOM', 'HAL', 'HES', 'KMI', 'MRO', 'MPC', 'OXY',
               'OKE', 'PSX', 'PXD', 'SLB', 'VLO', 'WMB']
}

In [24]:
ativos = []
for setor, empresas in setores.items():
    ativos.extend(empresas)
    
ativos.sort()

## Download dos Dados

In [25]:
if downloadData:

    start_date = "2017-10-21"
    end_date = "2022-10-21"

    data = web.DataReader(name = '^GSPC', data_source = 'yahoo', start = start_date, end = end_date)
    SP500_index = pd.DataFrame(data['Close']).reset_index().rename(columns={'Close': 'SP500', 'Date': 'Dia'})

    SP500_close = pd.DataFrame()

    for ativo in ativos:
  
        if ativo == 'BF.B':
            ativo = 'BF-B'

        if ativo == 'BRK.B':
            ativo = 'BRK-B'

        data = web.DataReader(name = ativo, data_source = 'yahoo', start = start_date, end = end_date)
        temp_close = pd.DataFrame(data['Close'])
        SP500_close = pd.concat([SP500_close, temp_close], axis = 1)

        
    SP500_close.columns = ativos
    SP500_close.reset_index(inplace = True)
    SP500_close.rename(columns={'Date': 'Dia'}, inplace = True)

    time = datetime.now().strftime('%d-%B-%Ih%Mmin')
    outdir = './data/{}'.format(time)

    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    SP500_close.to_csv(path_or_buf = os.path.join(outdir, 'SP500_close'), index = False)
    SP500_index.to_csv(path_or_buf = os.path.join(outdir, 'SP500_index'), index = False)
    
else:
    
    SP500_close = pd.read_csv('data/2017-10-21-2022-10-21/SP500_close')
    SP500_index = pd.read_csv('data/2017-10-21-2022-10-21/SP500_index')

## Pré-Processamento dos Dados

In [29]:
def generatorTimeframeTable(table, ativo):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    
    TimeframeTable = pd.DataFrame(np.zeros((len(table[ativo])-steps, steps+1), dtype='float64'), columns = nameColumns)

    for index, close in enumerate(table[ativo]):
        tempA = index
        tempB = 0
        for i in range(steps+1):
            if tempA < len(table[ativo])-steps and tempA >=0:
                TimeframeTable.iloc[tempA, tempB] = close

            tempA -= 1
            tempB += 1

    timeIndex = table.iloc[steps:,0]
    TimeframeTable["Dia"] = timeIndex.to_numpy()
    TimeframeTable.set_index("Dia", inplace = True)
    
    return TimeframeTable

In [30]:
def createTrainScaler(df):
    
    trainScaler = pd.DataFrame()
 
    for _ in range(steps+1):
        temp_close = pd.DataFrame(df.iloc[:,-1])
        trainScaler = pd.concat([trainScaler, temp_close], axis = 1)

    scaler = StandardScaler()

    scaler.fit(trainScaler)

    return scaler

In [31]:
def preprocessingdata(steps, df, ativos):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    

    aux = []
    
    for ativo in ativos:
        trainDataAtivo = generatorTimeframeTable(df, ativo)
        trainDataAtivo.dropna(axis = 0, inplace = True)
        
        #----Score-Z--------------------------------------
        scaler = createTrainScaler(trainDataAtivo)
        trainDataAtivo = scaler.transform(trainDataAtivo)
        #-------------------------------------------------
        aux.append(trainDataAtivo)
    
    trainData = np.concatenate(tuple(aux), axis=0)
    
    X = trainData[:, :-1]
    y = trainData[:, -1]
    

    #------Divisão de dados entre Treino e Validação------------------------------------------------
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, shuffle = False)

    X_treino = X_treino.reshape((-1, steps, 1))
    X_teste = X_teste.reshape((-1, steps, 1))
    #-----------------------------------------------------------------------------------------------
    
    return [X_treino, X_teste, y_treino, y_teste]

## Construção, Treinamento e Avaliação do Modelo

In [32]:
# Callbacks

tensorboard = TensorBoard(log_dir="logs/{}".format(datetime.now().strftime('%d-%B-%Ih%Mmin')))

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=20,
                          verbose = verbose,
                          restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='loss',
                              factor=0.2,
                              patience=3,
                              mode="min",
                              verbose = verbose,
                              min_delta=0.0001,
                              min_lr=0)

callbacks = [tensorboard, earlystop, reduce_lr, TerminateOnNaN()]

In [33]:
def create_model():
     
    model = Sequential()
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True,
                   input_shape = (steps, 1)))  

    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True))  
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = False)) 
    
    model.add(Dense(1, activation = 'linear'))
    
    Lmse = MeanSquaredError()
    
    opt_Adadelta = Adadelta(learning_rate = 0.01, rho = 0.95, epsilon = 1e-07)

    model.compile(loss= Lmse, optimizer = opt_Adadelta)

    return model

In [34]:
def fillTableFrame(ativo, tablePrevision, model, table = SP500_close):

    TimeframeTable = generatorTimeframeTable(table, ativo)
    
    index_data = TimeframeTable.index
    
    scaler = createTrainScaler(TimeframeTable)

    TimeframeTable = scaler.transform(TimeframeTable)
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))

    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    
    for day in forecast:
        
        current_info = TimeframeTable.iloc[-1, 1:].to_numpy()
        
        standardCurrentInfo = current_info.reshape(1, steps, 1).astype('float32')
        
        current_forecast = model.predict(standardCurrentInfo, verbose=False).reshape(1,)
        
        new_line = np.concatenate((current_info, current_forecast), axis = 0)
        
        TimeframeTable = pd.concat([TimeframeTable,
                                    pd.DataFrame(new_line.reshape(1, -1),
                                                 columns = nameColumns,
                                                 index = [day])], axis = 0)
        
        
    index_data = TimeframeTable.index  
    
    TimeframeTable = scaler.inverse_transform(TimeframeTable)
    
    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    TimeframeTable.index = pd.to_datetime(TimeframeTable.index)
    
    
    #--------Popula tabela de previsão---------------------------------------------------
    if ativo in ativos:
            for day in forecast:
                tablePrevision.loc[day, ativo] = TimeframeTable.loc[day, 'Close-0']
    #------------------------------------------------------------------------------------
   
    return TimeframeTable

In [35]:
# Popula tabela de log-Retorno, Previsão de Preços e gera gráficos

def fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model):
    
    lengthTable = len(tableLogRet)
    
    time = datetime.now().strftime('%d-%B-%Ih%Mmin')

    for ativo in setor:

        TimeframeSPAux = fillTableFrame(ativo, tablePrevision, model)

        #-----------Graphic------------------------------------------------------------------------------------------
        if graphic:
            
            outdir = './graphics/{}-{}'.format(nameSetor, time)

            if not os.path.exists(outdir):
                os.mkdir(outdir)
            
            fig, ax = plt.subplots()
            ax.plot(TimeframeSPAux.index[:-len(forecast)], TimeframeSPAux.iloc[:-len(forecast), -1], linewidth=3.0, c = 'b')
            ax.plot(TimeframeSPAux.index[-len(forecast):], TimeframeSPAux.iloc[-len(forecast):, -1], linewidth=3.0, c = 'c', ls = '-')
            ax.legend(['Atual', 'Previsão'])
            ax.set_title('Preço de Fechamento - {}'.format(ativo))
            ax.set(xlabel='Tempo (ano)', ylabel='Preço ($)')
            nameGraphic = '{}.jpg'.format(ativo)
            fullname = os.path.join(outdir, nameGraphic)
            plt.savefig(fullname)
            plt.close(fig)
        #------------------------------------------------------------------------------------------------------------

        
        #--------Popula tabela de Log Retorno------------------------------------------------------------------------
        for n in range(len(forecast)):
            tableLogRet.loc[lengthTable-n-1, ativo] = \
            np.log(TimeframeSPAux.iloc[lengthTable-steps-n-1, -1] / TimeframeSPAux.iloc[lengthTable-steps-n-1-logRetPeriod, -1])
        #------------------------------------------------------------------------------------------------------------

In [36]:
# Cria tabela de Previsão

update = pd.DataFrame(index = pd.to_datetime(forecast), columns = ativos) \
    .reset_index().rename(columns={'index': 'Dia'})

tablePrevision = pd.concat([SP500_close, update], axis = 0, ignore_index = True).set_index('Dia')

tablePrevision.index = pd.to_datetime(tablePrevision.index)

In [37]:
# Cria tabela de Log-Retorno vazia 

index_data = pd.to_datetime(SP500_close['Dia'].append(pd.Series(forecast)))

tableLogRet = pd.DataFrame(index = index_data,
                           columns = ativos).reset_index().rename(columns={'index': 'Dia'})

tableLogRet['Dia'] = tableLogRet['Dia'].apply(lambda date: date.strftime('%d/%m'))

In [40]:
for nameSetor, setor in setores.items():

    if trainModel:
        X_treino, X_teste, y_treino, y_teste = preprocessingdata(steps, SP500_close, setor)
        model = create_model()

        model.fit(x = X_treino,
                  y = y_treino,
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = verbose,
                  validation_data = (X_teste, y_teste),
                  callbacks = callbacks)
        
        #---------Save Model---------------------------
        model.save('saveModel/{}/'.format(nameSetor),
                   overwrite=True,
                   include_optimizer=True,
                   save_format = 'tf')
        #----------------------------------------------

        scoreTrain = model.evaluate(X_treino, y_treino)
        scoreTest = model.evaluate(X_teste, y_teste)

        print('\n\nErro quadrático médio em dados de treinamento: {:.5f}\n\nErro quadrático médio em dados de teste: {:.5f}\n\n'\
            .format(scoreTrain, scoreTest))

        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)
        
    else:
        model = load_model('saveModel/{}/'.format(nameSetor))
        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)


Epoch 1/2000
675/675 - 22s - loss: 0.1800 - val_loss: 0.0889 - lr: 0.0100 - 22s/epoch - 33ms/step
Epoch 2/2000
675/675 - 12s - loss: 0.0758 - val_loss: 0.0667 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 3/2000
675/675 - 12s - loss: 0.0601 - val_loss: 0.0570 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 4/2000
675/675 - 12s - loss: 0.0532 - val_loss: 0.0523 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 5/2000
675/675 - 12s - loss: 0.0498 - val_loss: 0.0487 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 6/2000
675/675 - 12s - loss: 0.0472 - val_loss: 0.0467 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 7/2000
675/675 - 12s - loss: 0.0449 - val_loss: 0.0438 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 8/2000
675/675 - 12s - loss: 0.0428 - val_loss: 0.0418 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 9/2000
675/675 - 12s - loss: 0.0409 - val_loss: 0.0399 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 10/2000
675/675 - 12s - loss: 0.0392 - val_loss: 0.0385 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 11/2000
675/6

Epoch 84/2000
675/675 - 12s - loss: 0.0154 - val_loss: 0.0154 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 85/2000
675/675 - 12s - loss: 0.0153 - val_loss: 0.0154 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 86/2000
675/675 - 12s - loss: 0.0152 - val_loss: 0.0153 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 87/2000
675/675 - 12s - loss: 0.0151 - val_loss: 0.0152 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 88/2000
675/675 - 12s - loss: 0.0151 - val_loss: 0.0151 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 89/2000
675/675 - 12s - loss: 0.0150 - val_loss: 0.0150 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 90/2000
675/675 - 12s - loss: 0.0149 - val_loss: 0.0149 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 91/2000
675/675 - 12s - loss: 0.0148 - val_loss: 0.0149 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 92/2000
675/675 - 12s - loss: 0.0147 - val_loss: 0.0147 - lr: 0.0100 - 12s/epoch - 17ms/step
Epoch 93/2000
675/675 - 12s - loss: 0.0146 - val_loss: 0.0148 - lr: 0.0100 - 12s/epoch - 18ms/step
Epoch 94/2

Epoch 166/2000
675/675 - 12s - loss: 0.0111 - val_loss: 0.0113 - lr: 0.0020 - 12s/epoch - 18ms/step
Epoch 167/2000

Epoch 167: ReduceLROnPlateau reducing learning rate to 0.0003999999724328518.
675/675 - 12s - loss: 0.0111 - val_loss: 0.0113 - lr: 0.0020 - 12s/epoch - 17ms/step
Epoch 168/2000
675/675 - 12s - loss: 0.0111 - val_loss: 0.0113 - lr: 4.0000e-04 - 12s/epoch - 17ms/step
Epoch 169/2000
675/675 - 12s - loss: 0.0111 - val_loss: 0.0113 - lr: 4.0000e-04 - 12s/epoch - 17ms/step
Epoch 170/2000

Epoch 170: ReduceLROnPlateau reducing learning rate to 7.999999215826393e-05.
675/675 - 12s - loss: 0.0111 - val_loss: 0.0113 - lr: 4.0000e-04 - 12s/epoch - 17ms/step
Epoch 171/2000
675/675 - 12s - loss: 0.0111 - val_loss: 0.0113 - lr: 8.0000e-05 - 12s/epoch - 18ms/step
Epoch 172/2000
675/675 - 12s - loss: 0.0111 - val_loss: 0.0113 - lr: 8.0000e-05 - 12s/epoch - 17ms/step
Epoch 173/2000

Epoch 173: ReduceLROnPlateau reducing learning rate to 1.599999814061448e-05.
675/675 - 13s - loss: 0.0111

INFO:tensorflow:Assets written to: saveModel/Energy/assets


INFO:tensorflow:Assets written to: saveModel/Energy/assets


169/169 [==============================] - 1s 7ms/step - loss: 0.0113


Erro quadrático médio em dados de treinamento: 0.01110

Erro quadrático médio em dados de teste: 0.01132


Epoch 1/2000
2054/2054 - 45s - loss: 0.1049 - val_loss: 0.0603 - lr: 0.0100 - 45s/epoch - 22ms/step
Epoch 2/2000
2054/2054 - 35s - loss: 0.0520 - val_loss: 0.0494 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 3/2000
2054/2054 - 35s - loss: 0.0446 - val_loss: 0.0434 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 4/2000
2054/2054 - 36s - loss: 0.0391 - val_loss: 0.0386 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 5/2000
2054/2054 - 35s - loss: 0.0349 - val_loss: 0.0353 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 6/2000
2054/2054 - 36s - loss: 0.0318 - val_loss: 0.0326 - lr: 0.0100 - 36s/epoch - 18ms/step
Epoch 7/2000
2054/2054 - 37s - loss: 0.0298 - val_loss: 0.0308 - lr: 0.0100 - 37s/epoch - 18ms/step
Epoch 8/2000
2054/2054 - 35s - loss: 0.0282 - val_loss: 0.0294 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 9/2000
2054/205

Epoch 80/2000
2054/2054 - 35s - loss: 0.0105 - val_loss: 0.0115 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 81/2000
2054/2054 - 35s - loss: 0.0105 - val_loss: 0.0114 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 82/2000
2054/2054 - 35s - loss: 0.0105 - val_loss: 0.0113 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 83/2000
2054/2054 - 35s - loss: 0.0104 - val_loss: 0.0113 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 84/2000
2054/2054 - 35s - loss: 0.0104 - val_loss: 0.0114 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 85/2000
2054/2054 - 36s - loss: 0.0104 - val_loss: 0.0112 - lr: 0.0100 - 36s/epoch - 18ms/step
Epoch 86/2000
2054/2054 - 36s - loss: 0.0103 - val_loss: 0.0112 - lr: 0.0100 - 36s/epoch - 18ms/step
Epoch 87/2000
2054/2054 - 35s - loss: 0.0103 - val_loss: 0.0112 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 88/2000
2054/2054 - 36s - loss: 0.0103 - val_loss: 0.0112 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 89/2000

Epoch 89: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.

INFO:tensorflow:Assets written to: saveModel/Financials/assets


INFO:tensorflow:Assets written to: saveModel/Financials/assets


514/514 [==============================] - 4s 7ms/step - loss: 0.0110


Erro quadrático médio em dados de treinamento: 0.01014

Erro quadrático médio em dados de teste: 0.01104




# Resultados

In [41]:
tableLogRet

,Dia,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
0,24/10,-0.028104,-0.070165,0.009660,-0.097270,0.002473,0.005414,-0.013053,-0.023760,-0.071460,...,-0.137958,-0.230894,0.051548,-0.093068,-0.104269,-0.071549,-0.047608,-0.103562,-0.051527,-0.067073
1,25/10,-0.005556,-0.047145,0.021813,-0.113248,0.008050,0.015086,-0.013310,-0.004750,-0.049512,...,-0.150742,-0.215916,0.072632,-0.051262,-0.087136,-0.061975,-0.032195,-0.083463,-0.105818,-0.039219
2,26/10,0.010191,0.010363,0.023840,-0.091805,0.016371,0.012493,0.013720,0.013773,-0.027337,...,-0.075544,-0.201938,0.045874,-0.040985,-0.069650,-0.036896,-0.015892,-0.078709,-0.152533,-0.026923
3,27/10,0.028459,0.052098,0.032720,-0.084410,-0.005042,0.004940,0.036605,0.011346,-0.012944,...,-0.000355,-0.201103,0.012522,0.007024,-0.038498,-0.020342,0.006086,-0.033439,-0.160053,-0.014552
4,28/10,0.036344,0.093614,0.055926,-0.068235,-0.007491,0.024106,0.042770,0.016374,-0.002916,...,0.017530,-0.188726,0.027623,-0.001207,-0.027812,-0.019185,0.024322,-0.014636,-0.152814,-0.018454
5,31/10,0.045507,0.124326,0.074900,-0.069606,0.004067,0.029380,0.053558,0.025336,0.006397,...,-0.091738,-0.165874,0.007751,0.013968,-0.016350,-0.006185,0.035285,-0.006605,-0.179427,-0.009033
6,01/11,0.050406,0.092714,0.065926,-0.075277,-0.000356,0.035336,0.043985,0.040730,0.010195,...,-0.087953,-0.142530,0.071612,0.030471,-0.010561,0.005947,0.038960,-0.003777,-0.153670,-0.014755
7,02/11,0.032255,0.057506,0.042218,-0.061538,-0.020733,0.013072,0.019900,0.036667,-0.012344,...,-0.092810,-0.149421,0.062182,0.020829,-0.035924,0.002183,0.013752,-0.028148,-0.140438,-0.033945
8,03/11,0.039415,0.101251,0.065728,-0.010038,-0.007575,0.029612,0.023758,0.040842,0.001055,...,-0.058066,-0.095021,0.076603,0.047070,-0.009049,0.010371,0.017818,-0.006753,-0.118040,-0.025269
9,04/11,0.048674,0.122385,0.087184,0.021729,0.053954,0.034543,0.037326,0.056410,0.004693,...,-0.038301,-0.072026,0.086579,0.079956,-0.001549,0.018575,0.042375,0.006645,-0.104398,-0.007140


### Salva Métricas

In [42]:
# Salva tabela de previsão

outdirP = './previsao/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirP):
    os.mkdir(outdirP)

fullnameP = os.path.join(outdirP, 'previsao.csv')

tablePrevision.to_csv(fullnameP, index = True, decimal = '.', sep=',')  

In [43]:
# Salva tabela Entregável do Log-Retorno no padrão

outdirLR = './logRetorno/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirLR):
    os.mkdir(outdirLR)

fullnameLR = os.path.join(outdirLR, 'logRetorno.csv')

tableLogRet.iloc[-len(forecast):, :].to_csv(fullnameLR, index = False, decimal = '.', sep=',')       